In [1]:
import numpy as np
import cv2
import sys, os, time, argparse, logging
import simplejson
import argparse
import math

import mylib.io as myio
from mylib.displays import drawActionResult
import mylib.funcs as myfunc
import mylib.feature_proc as myproc 
from mylib.action_classifier import ClassifierOnlineTest
from mylib.action_classifier import * # Import sklearn related libraries

# CURR_PATH = os.path.dirname(os.path.abspath(__file__))+"/" # ? curr_path
DRAW_FPS = True


folder_suffix = "1"
DO_INFER_ACTIONS =  True
SAVE_RESULTANT_SKELETON_TO_TXT_AND_IMAGE = True
def set_source_images_from_folder():
    return "images", 0
SRC_IMAGE_FOLDER, SKIP_NUM_IMAGES = set_source_images_from_folder()
SKIP_NUM_IMAGES

0

In [2]:
folder_suffix += SRC_IMAGE_FOLDER.split('/')[0] #[-2] # plus folder name
folder_suffix

'4images'

# image_size = "304x240"
# image_size = "240x208"
# OpenPose_MODEL = ["mobilenet_thin", "cmu"][1]
image_size = "432x368" # 10 fps
OpenPose_MODEL = ["mobilenet_thin", "cmu"][0]
    

if DO_INFER_ACTIONS:
    LOAD_MODEL_PATH = "../model/trained_classifier.pickle"
    action_labels=  ['jump','kick','punch','run','sit','squat','stand','walk','wave']

    

if SAVE_RESULTANT_SKELETON_TO_TXT_AND_IMAGE:
    SKELETON_FOLDER = "skeleton_data/"
    SAVE_DETECTED_SKELETON_TO =         "skeleton_data/skeletons"+folder_suffix+"/"
    SAVE_DETECTED_SKELETON_IMAGES_TO =  "skeleton_data/skeletons"+folder_suffix+"_images/"
    SAVE_IMAGES_INFO_TO =             "skeleton_data/images_info"+folder_suffix+".txt"

    # create folders for saving results
    if not os.path.exists(SKELETON_FOLDER):
        os.makedirs(SKELETON_FOLDER)
    if not os.path.exists(SAVE_DETECTED_SKELETON_TO):
        os.makedirs(SAVE_DETECTED_SKELETON_TO)
    if not os.path.exists(SAVE_DETECTED_SKELETON_IMAGES_TO):
        os.makedirs(SAVE_DETECTED_SKELETON_IMAGES_TO)

# Openpose include files and configs ==============================================================

sys.path.append("githubs/tf-pose-estimation")
from tf_pose.networks import get_graph_path, model_wh
from tf_pose.estimator import TfPoseEstimator
from tf_pose import common

logger = logging.getLogger('TfPoseEstimator')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

# ---- For tf 1.13.1, The following setting is needed
import tensorflow as tf
from tensorflow import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# If GPU memory is small, modify the MAX_FRACTION_OF_GPU_TO_USE
MAX_FRACTION_OF_GPU_TO_USE = 0.5
config.gpu_options.per_process_gpu_memory_fraction=MAX_FRACTION_OF_GPU_TO_USE

# Openpose Human pose detection ==============================================================

class SkeletonDetector(object):
    # This func is mostly copied from https://github.com/ildoonet/tf-pose-estimation

    def __init__(self, model=None, image_size=None):
        
        if model is None:
            model = "cmu"

        if image_size is None:
            image_size = "432x368" # 7 fps
            # image_size = "336x288"
            # image_size = "304x240" # 14 fps

        models = set({"mobilenet_thin", "cmu"})
        self.model = model if model in models else "mobilenet_thin"
        # parser = argparse.ArgumentParser(description='tf-pose-estimation run')
        # parser.add_argument('--image', type=str, default='./images/p1.jpg')
        # parser.add_argument('--model', type=str, default='cmu', help='cmu / mobilenet_thin')

        # parser.add_argument('--resize', type=str, default='0x0',
        #                     help='if provided, resize images before they are processed. default=0x0, Recommends : 432x368 or 656x368 or 1312x736 ')
        # parser.add_argument('--resize-out-ratio', type=float, default=4.0,
        #                     help='if provided, resize heatmaps before they are post-processed. default=1.0')
        self.resize_out_ratio = 4.0

        # args = parser.parse_args()

        w, h = model_wh(image_size)
        if w == 0 or h == 0:
            e = TfPoseEstimator(
                    get_graph_path(self.model),
                    target_size=(432, 368),
                    tf_config=config)
        else:
            e = TfPoseEstimator(
                get_graph_path(self.model), 
                target_size=(w, h),
                tf_config=config)

        # self.args = args
        self.w, self.h = w, h
        self.e = e
        self.fps_time = time.time()
        self.cnt_image = 0

    def detect(self, image):
        self.cnt_image += 1
        if self.cnt_image == 1:
            self.image_h = image.shape[0]
            self.image_w = image.shape[1]
            self.scale_y = 1.0 * self.image_h / self.image_w
        t = time.time()

        # Inference
        humans = self.e.inference(image, resize_to_default=(self.w > 0 and self.h > 0),
                                #   upsample_size=self.args.resize_out_ratio)
                                  upsample_size=self.resize_out_ratio)

        # Print result and time cost
        elapsed = time.time() - t
        logger.info('inference image in %.4f seconds.' % (elapsed))

        return humans
    
    def draw(self, img_disp, humans):
        img_disp = TfPoseEstimator.draw_humans(img_disp, humans, imgcopy=False)
        # logger.debug('show+')
        if DRAW_FPS:
            cv2.putText(img_disp,
                        # "Processing speed: {:.1f} fps".format( (1.0 / (time.time() - self.fps_time) )),
                        "fps = {:.1f}".format( (1.0 / (time.time() - self.fps_time) )),
                        (10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 255), 2)
        self.fps_time = time.time()

    def humans_to_skelsList(self, humans, scale_y = None): # get (x, y * scale_y)
        # type: humans: returned from self.detect()
        # rtype: list[list[]]
        if scale_y is None:
            scale_y = self.scale_y
        skeletons = []
        NaN = 0
        for human in humans:
            skeleton = [NaN]*(18*2)
            for i, body_part in human.body_parts.items(): # iterate dict
                idx = body_part.part_idx
                skeleton[2*idx]=body_part.x
                skeleton[2*idx+1]=body_part.y * scale_y
            skeletons.append(skeleton)
            print(100*'@')
            print(skeletons,len(skeletons))
        return skeletons, scale_y
    


# ==============================================================

def add_white_region_to_left_of_image(image_disp):
    r, c, d = image_disp.shape
    blank = 255 + np.zeros((r, int(c/4), d), np.uint8)
    image_disp = np.hstack((blank, image_disp))
    return image_disp

def remove_skeletons_with_few_joints(skeletons):
    good_skeletons = []
    for skeleton in skeletons:
        px = skeleton[2:2+13*2:2]
        py = skeleton[3:2+13*2:2]
        num_valid_joints = len([x for x in px if x!=0])
        num_leg_joints = len([x for x in px[-6:] if x!=0])
        total_size = max(py) - min(py)
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # IF JOINTS ARE MISSING, TRY CHANGING THESE VALUES:
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        print('num_valid_joints',num_valid_joints)
        if num_valid_joints >= 5 and total_size >= 0.1 and num_leg_joints >= 2: 
            good_skeletons.append(skeleton) # add this skeleton only when all requirements are satisfied
    return good_skeletons

class MultiPersonClassifier(object):
    def __init__(self, LOAD_MODEL_PATH, action_labels):
        self.create_classifier = lambda human_id: ClassifierOnlineTest(
            LOAD_MODEL_PATH, action_types = action_labels, human_id=human_id)
        self.dict_id2clf = {} # human id -> classifier of this person

    def classify(self, dict_id2skeleton):

        # Clear people not in view
        old_ids = set(self.dict_id2clf)
        cur_ids = set(dict_id2skeleton)
        humans_not_in_view = list(old_ids - cur_ids)
        for human in humans_not_in_view:
            del self.dict_id2clf[human]

        # Predict each person's action
        id2label = {}
        print(100*'&')
        for id, skeleton in dict_id2skeleton.items():
            
            if id not in self.dict_id2clf: # add this new person
                self.dict_id2clf[id] = self.create_classifier(id)
                print('self.dict_id2clf',len(self.dict_id2clf))
                print('self.dict_id2clf[id]',self.dict_id2clf[id])
            
            classifier = self.dict_id2clf[id]
            print('classifier',classifier)
            id2label[id] = classifier.predict(skeleton) # predict label
            print(100*'*')
            print("\n\nPredicting label for human{}".format(id))
            print("  skeleton: {}".format(skeleton))
            print("  label: {}".format(id2label[id]))

        return id2label

    def get(self, id):
        # type: id: int or "min"
        if len(self.dict_id2clf) == 0:
            return None 
        if id == 'min':
            id = min(self.dict_id2clf.keys())
        return self.dict_id2clf[id]

W0810 09:17:11.978364 140636630144768 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/mobilenet/mobilenet.py:369: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [3]:
my_detector = SkeletonDetector(OpenPose_MODEL, image_size)
print(my_detector)

[2019-08-10 09:17:11,985] [TfPoseEstimator] [INFO] loading graph from /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
[2019-08-10 09:17:11,985] [TfPoseEstimator] [INFO] loading graph from /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
I0810 09:17:11.985104 140636630144768 estimator.py:310] loading graph from /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
W0810 09:17:11.986301 140636630144768 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0810 09:17:11.986606 140636630144768 deprecation_wrapper.py:119] From /home/anht1/Desktop/TESTCODEGIT/realtimean/src/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef in

TfPoseEstimator/MobilenetV1/Conv2d_0/weights
TfPoseEstimator/image
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_0/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_0/Relu
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Relu
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D_bn_offset
TfPoseEstimator/M

In [4]:
images_loader = myio.DataLoader_folder(SRC_IMAGE_FOLDER, SKIP_NUM_IMAGES)
images_loader

In [5]:
if DO_INFER_ACTIONS:
        multipeople_classifier = MultiPersonClassifier(LOAD_MODEL_PATH, action_labels)
multiperson_tracker = myfunc.Tracker()
multiperson_tracker

In [6]:
# -- Loop through all images
ith_img = 1
while ith_img <= images_loader.num_images:
    img, img_action_type, img_info = images_loader.load_next_image()
    image_disp = img.copy()

    print("\n\n========================================")
    print("\nProcessing {}/{}th image\n".format(ith_img, images_loader.num_images))

    # -- Detect all people's skeletons
    humans = my_detector.detect(img)
    print(humans)
    skeletons, scale_y = my_detector.humans_to_skelsList(humans)
    skeletons = remove_skeletons_with_few_joints(skeletons)
    print(skeletons,len(skeletons))

    # -- Track people
    dict_id2skeleton = multiperson_tracker.track(skeletons) # int id -> np.array() skeleton
    print(dict_id2skeleton,len(dict_id2skeleton))
    # -- Recognize action for each person
    if len(dict_id2skeleton):
        print('dict_id2skeleton',len(dict_id2skeleton))
        if DO_INFER_ACTIONS:
            min_id = min(dict_id2skeleton.keys())
            print('min_id',min_id)
            dict_id2label = multipeople_classifier.classify(dict_id2skeleton)
            print("dict_id2label",dict_id2label)
            print("prediced label is :", dict_id2label[min_id])
        else: # reserve only one skeleton
            min_id = min(dict_id2skeleton.keys()) 
            dict_id2skeleton = {min_id : dict_id2skeleton[min_id]}
            dict_id2label = {min_id : img_action_type}
            print("Ground_truth label is :", dict_id2label[min_id])

    # -- Draw
    my_detector.draw(image_disp, humans) # Draw all skeletons

    if len(dict_id2skeleton): 

        # Draw outer box and label for each person 
        for id, label in dict_id2label.items():
            skeleton = dict_id2skeleton[id]
            skeleton[1::2] = skeleton[1::2] / scale_y # scale the y data back to original
            print("Drawing skeleton: ", dict_id2skeleton[id], "with label:", label, ".")
            drawActionResult(image_disp, id, skeleton, label)


    # Add blank to the left for displaying prediction scores of each class
    image_disp = add_white_region_to_left_of_image(image_disp)

    # Draw predicting score for only 1 person (not using for)
    if DO_INFER_ACTIONS and len(dict_id2skeleton):
        multipeople_classifier.get(id='min').draw_scores_onto_image(image_disp)


    # -- Write skeleton.txt and image.png
    if SAVE_RESULTANT_SKELETON_TO_TXT_AND_IMAGE:

        ids = sorted(dict_id2skeleton.keys())
        skel_to_save = [img_info + dict_id2skeleton[id].tolist() for id in ids]


        myio.save_skeletons(SAVE_DETECTED_SKELETON_TO 
            + myfunc.int2str(ith_img, 5) + ".txt", skel_to_save)
        cv2.imwrite(SAVE_DETECTED_SKELETON_IMAGES_TO 
            + myfunc.int2str(ith_img, 5) + ".png", image_disp)



    # -- Display
    if 1:
        if ith_img == 1:
            window_name = "action_recognition"
            cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
        # image_disp = cv2.resize(image_disp, (0,0), fx=1.5, fy=1.5) # resize to make picture bigger
        cv2.imshow(window_name, image_disp)
        q = cv2.waitKey(1)
        if q != -1 and chr(q) == 'q':
            break

    # -- Loop
    print("\n")
    ith_img += 1

[2019-08-10 09:17:13,614] [TfPoseEstimator] [DEBUG] inference+ original shape=500x500
[2019-08-10 09:17:13,614] [TfPoseEstimator] [DEBUG] inference+ original shape=500x500
I0810 09:17:13.614025 140636630144768 estimator.py:549] inference+ original shape=500x500
[2019-08-10 09:17:13,640] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:13,640] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
I0810 09:17:13.640032 140636630144768 estimator.py:561] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:13,643] [TfPoseEstimator] [DEBUG] estimate time=0.00203
[2019-08-10 09:17:13,643] [TfPoseEstimator] [DEBUG] estimate time=0.00203
I0810 09:17:13.643577 140636630144768 estimator.py:565] estimate time=0.00203
[2019-08-10 09:17:13,644] [TfPoseEstimator] [INFO] inference image in 0.0307 seconds.
[2019-08-10 09:17:13,644] [TfPoseEstimator] [INFO] inference image in 0.0307 seconds.
I0810 09:17:13.644581 140636630144768 <ipython-input-2




Processing 1/16th image

[BodyPart:0-(0.50, 0.24) score=0.93 BodyPart:1-(0.50, 0.34) score=0.69 BodyPart:2-(0.43, 0.33) score=0.73 BodyPart:3-(0.40, 0.43) score=0.18 BodyPart:5-(0.57, 0.33) score=0.68 BodyPart:6-(0.63, 0.44) score=0.75 BodyPart:7-(0.55, 0.58) score=0.28 BodyPart:8-(0.44, 0.63) score=0.35 BodyPart:9-(0.37, 0.96) score=0.11 BodyPart:11-(0.51, 0.61) score=0.32 BodyPart:12-(0.56, 0.72) score=0.42 BodyPart:14-(0.48, 0.22) score=0.88 BodyPart:15-(0.52, 0.22) score=0.93 BodyPart:16-(0.46, 0.23) score=0.57 BodyPart:17-(0.56, 0.23) score=0.48]
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0.5046296296296297, 0.24456521739130435, 0.49537037037037035, 0.3423913043478261, 0.42592592592592593, 0.33152173913043476, 0.39814814814814814, 0.43478260869565216, 0, 0, 0.5694444444444444, 0.33152173913043476, 0.6342592592592593, 0.44021739130434784, 0.5462962962962963, 0.5760869565217391, 0.4444444444444444, 0.6304347826086957, 0.

[2019-08-10 09:17:13,829] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:13,829] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
I0810 09:17:13.829944 140636630144768 estimator.py:561] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:13,833] [TfPoseEstimator] [DEBUG] estimate time=0.00222
[2019-08-10 09:17:13,833] [TfPoseEstimator] [DEBUG] estimate time=0.00222
I0810 09:17:13.833243 140636630144768 estimator.py:565] estimate time=0.00222
[2019-08-10 09:17:13,834] [TfPoseEstimator] [INFO] inference image in 0.0226 seconds.
[2019-08-10 09:17:13,834] [TfPoseEstimator] [INFO] inference image in 0.0226 seconds.
I0810 09:17:13.834264 140636630144768 <ipython-input-2-364c885c4945>:121] inference image in 0.0226 seconds.
[2019-08-10 09:17:13,859] [TfPoseEstimator] [DEBUG] inference+ original shape=500x500
[2019-08-10 09:17:13,859] [TfPoseEstimator] [DEBUG] inference+ original shape=500x500
I0810 09:17:13.859138 1406366301447

[BodyPart:0-(0.50, 0.24) score=0.93 BodyPart:1-(0.50, 0.34) score=0.69 BodyPart:2-(0.43, 0.33) score=0.73 BodyPart:3-(0.40, 0.43) score=0.18 BodyPart:5-(0.57, 0.33) score=0.68 BodyPart:6-(0.63, 0.44) score=0.75 BodyPart:7-(0.55, 0.58) score=0.28 BodyPart:8-(0.44, 0.63) score=0.35 BodyPart:9-(0.37, 0.96) score=0.11 BodyPart:11-(0.51, 0.61) score=0.32 BodyPart:12-(0.56, 0.72) score=0.42 BodyPart:14-(0.48, 0.22) score=0.88 BodyPart:15-(0.52, 0.22) score=0.93 BodyPart:16-(0.46, 0.23) score=0.57 BodyPart:17-(0.56, 0.23) score=0.48]
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0.5046296296296297, 0.24456521739130435, 0.49537037037037035, 0.3423913043478261, 0.42592592592592593, 0.33152173913043476, 0.39814814814814814, 0.43478260869565216, 0, 0, 0.5694444444444444, 0.33152173913043476, 0.6342592592592593, 0.44021739130434784, 0.5462962962962963, 0.5760869565217391, 0.4444444444444444, 0.6304347826086957, 0.37037037037037035, 0.9565217

[2019-08-10 09:17:14,050] [TfPoseEstimator] [DEBUG] inference+ original shape=854x480
[2019-08-10 09:17:14,050] [TfPoseEstimator] [DEBUG] inference+ original shape=854x480
I0810 09:17:14.050992 140636630144768 estimator.py:549] inference+ original shape=854x480
[2019-08-10 09:17:14,068] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:14,068] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
I0810 09:17:14.068999 140636630144768 estimator.py:561] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:14,072] [TfPoseEstimator] [DEBUG] estimate time=0.00190
[2019-08-10 09:17:14,072] [TfPoseEstimator] [DEBUG] estimate time=0.00190
I0810 09:17:14.072049 140636630144768 estimator.py:565] estimate time=0.00190
[2019-08-10 09:17:14,073] [TfPoseEstimator] [INFO] inference image in 0.0223 seconds.
[2019-08-10 09:17:14,073] [TfPoseEstimator] [INFO] inference image in 0.0223 seconds.
I0810 09:17:14.073309 140636630144768 <ipython-input-2






Processing 8/16th image

[BodyPart:0-(0.28, 0.16) score=0.85 BodyPart:1-(0.31, 0.27) score=0.76 BodyPart:2-(0.28, 0.27) score=0.74 BodyPart:3-(0.28, 0.38) score=0.11 BodyPart:5-(0.35, 0.27) score=0.72 BodyPart:6-(0.38, 0.39) score=0.75 BodyPart:7-(0.38, 0.51) score=0.78 BodyPart:8-(0.29, 0.53) score=0.62 BodyPart:9-(0.29, 0.68) score=0.71 BodyPart:10-(0.29, 0.84) score=0.60 BodyPart:11-(0.34, 0.52) score=0.58 BodyPart:12-(0.36, 0.68) score=0.55 BodyPart:13-(0.38, 0.85) score=0.37 BodyPart:14-(0.28, 0.15) score=0.62 BodyPart:15-(0.30, 0.15) score=0.76 BodyPart:17-(0.32, 0.16) score=0.97, BodyPart:0-(0.49, 0.17) score=0.85 BodyPart:1-(0.51, 0.27) score=0.80 BodyPart:2-(0.47, 0.27) score=0.74 BodyPart:5-(0.55, 0.27) score=0.84 BodyPart:6-(0.56, 0.40) score=0.66 BodyPart:7-(0.56, 0.48) score=0.26 BodyPart:8-(0.48, 0.48) score=0.46 BodyPart:9-(0.49, 0.72) score=0.60 BodyPart:10-(0.48, 0.88) score=0.34 BodyPart:11-(0.53, 0.48) score=0.44 BodyPart:12-(0.56, 0.72) score=0.69 BodyPart:13-(

[2019-08-10 09:17:14,247] [TfPoseEstimator] [INFO] inference image in 0.0238 seconds.
I0810 09:17:14.247575 140636630144768 <ipython-input-2-364c885c4945>:121] inference image in 0.0238 seconds.
[2019-08-10 09:17:14,278] [TfPoseEstimator] [DEBUG] inference+ original shape=217x232
[2019-08-10 09:17:14,278] [TfPoseEstimator] [DEBUG] inference+ original shape=217x232
I0810 09:17:14.278317 140636630144768 estimator.py:549] inference+ original shape=217x232
[2019-08-10 09:17:14,296] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:14,296] [TfPoseEstimator] [DEBUG] inference- heatMat=216x184 pafMat=216x184
I0810 09:17:14.296653 140636630144768 estimator.py:561] inference- heatMat=216x184 pafMat=216x184
[2019-08-10 09:17:14,299] [TfPoseEstimator] [DEBUG] estimate time=0.00127
[2019-08-10 09:17:14,299] [TfPoseEstimator] [DEBUG] estimate time=0.00127
I0810 09:17:14.299051 140636630144768 estimator.py:565] estimate time=0.00127
[2019-08-10 09:17:14,299] [TfPo

[BodyPart:0-(0.28, 0.16) score=0.85 BodyPart:1-(0.31, 0.27) score=0.76 BodyPart:2-(0.28, 0.27) score=0.74 BodyPart:3-(0.28, 0.38) score=0.11 BodyPart:5-(0.35, 0.27) score=0.72 BodyPart:6-(0.38, 0.39) score=0.75 BodyPart:7-(0.38, 0.51) score=0.78 BodyPart:8-(0.29, 0.53) score=0.62 BodyPart:9-(0.29, 0.68) score=0.71 BodyPart:10-(0.29, 0.84) score=0.60 BodyPart:11-(0.34, 0.52) score=0.58 BodyPart:12-(0.36, 0.68) score=0.55 BodyPart:13-(0.38, 0.85) score=0.37 BodyPart:14-(0.28, 0.15) score=0.62 BodyPart:15-(0.30, 0.15) score=0.76 BodyPart:17-(0.32, 0.16) score=0.97, BodyPart:0-(0.49, 0.17) score=0.85 BodyPart:1-(0.51, 0.27) score=0.80 BodyPart:2-(0.47, 0.27) score=0.74 BodyPart:5-(0.55, 0.27) score=0.84 BodyPart:6-(0.56, 0.40) score=0.66 BodyPart:7-(0.56, 0.48) score=0.26 BodyPart:8-(0.48, 0.48) score=0.46 BodyPart:9-(0.49, 0.72) score=0.60 BodyPart:10-(0.48, 0.88) score=0.34 BodyPart:11-(0.53, 0.48) score=0.44 BodyPart:12-(0.56, 0.72) score=0.69 BodyPart:13-(0.56, 0.89) score=0.47 BodyPar


Mean score:
 [2.52033741e-06 4.78931159e-01 1.47271629e-13 9.65615555e-10
 4.99999062e-01 1.50411693e-08 8.52068703e-10 4.99023121e-14
 2.10672421e-02]
****************************************************************************************************


Predicting label for human1
  skeleton: [0.50925926 0.16304348 0.50462963 0.22282609 0.4212963  0.22282609
 0.36574074 0.2826087  0.27314815 0.32065217 0.59259259 0.2173913
 0.66666667 0.35326087 0.66203704 0.46195652 0.49537037 0.4673913
 0.47685185 0.60326087 0.52777778 0.82608696 0.61574074 0.46195652
 0.6712963  0.625      0.70833333 0.77717391 0.49537037 0.15217391
 0.52314815 0.14673913 0.45833333 0.14130435 0.5462963  0.13586957]
  label: 
dict_id2label {1: ''}
prediced label is : 
Drawing skeleton:  [0.50925926 0.16304348 0.50462963 0.22282609 0.4212963  0.22282609
 0.36574074 0.2826087  0.27314815 0.32065217 0.59259259 0.2173913
 0.66666667 0.35326087 0.66203704 0.46195652 0.49537037 0.4673913
 0.47685185 0.60326087 0.5277777